In [9]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools
import statistics

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [10]:
sp_df = pd.read_csv('SpotifyFeatures.csv')

In [11]:
sp_df['artist_name']=sp_df['artist_name'].str.upper()
sp_df['track_name']=sp_df['track_name'].str.upper()

In [12]:
sp_df

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,HENRI SALVADOR,C'EST BEAU DE FAIRE UN SHOW,0BRjO6ga9RKCKjfDqeFgWV,0,0.61100,0.389,99373,0.910,0.000000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,MARTIN & LES FÉES,PERDU D'AVANCE (PAR GAD ELMALEH),0BjC1NfoEOOusryehmNudP,1,0.24600,0.590,137373,0.737,0.000000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,JOSEPH WILLIAMS,DON'T LET ME BE LONELY TONIGHT,0CoSDzoNIKCRs124s9uTVy,3,0.95200,0.663,170267,0.131,0.000000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,HENRI SALVADOR,DIS-MOI MONSIEUR GORDON COOPER,0Gc6TVm52BwZD07Ki6tIvf,0,0.70300,0.240,152427,0.326,0.000000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,FABIEN NATAF,OUVERTURE,0IuslXpMROHdEPvSl1fTQK,4,0.95000,0.331,82625,0.225,0.123000,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232720,Soul,SLAVE,SON OF SLIDE,2XGLdVl7lGeq8ksM6Al7jT,39,0.00384,0.687,326240,0.714,0.544000,D,0.0845,-10.626,Major,0.0316,115.542,4/4,0.962
232721,Soul,JR THOMAS & THE VOLCANOS,BURNING FIRE,1qWZdkBl4UVPj9lK6HuuFM,38,0.03290,0.785,282447,0.683,0.000880,E,0.2370,-6.944,Minor,0.0337,113.830,4/4,0.969
232722,Soul,MUDDY WATERS,(I'M YOUR) HOOCHIE COOCHIE MAN,2ziWXUmQLrXTiYjCg2fZ2t,47,0.90100,0.517,166960,0.419,0.000000,D,0.0945,-8.282,Major,0.1480,84.135,4/4,0.813
232723,Soul,R.LUM.R,WITH MY WORDS,6EFsue2YbIG4Qkq8Zr9Rir,44,0.26200,0.745,222442,0.704,0.000000,A,0.3330,-7.137,Major,0.1460,100.031,4/4,0.489


In [ ]:
sp_df.dtypes

In [ ]:
sp_df.info()

In [ ]:
sp_df.shape

In [ ]:
sp_df['genre'].values[0]

In [ ]:
sp_df['artist_name'].values[0]

In [ ]:
sp_df[sp_df['artist_name']=='HARRY STYLES']

In [ ]:
sp_df[sp_df['id']=='5Q0Nhxo0l2bP3pNjpGJwV1'].nunique()

In [ ]:
id_genres = sp_df.groupby('id')['genre'].apply(list).reset_index()

In [ ]:
id_popularity = sp_df.groupby('id')['popularity'].apply(list).reset_index()

In [ ]:
id_genres

In [ ]:
id_popularity

In [ ]:
sp_df.drop(['genre','popularity'], axis=1, inplace=True)

In [ ]:
sp_df.drop_duplicates(inplace=True)

In [ ]:
sp_df=sp_df.merge(id_genres[['id','genre']], on = 'id',how = 'left')
sp_df=sp_df.merge(id_popularity[['id','popularity']], on = 'id',how = 'left')


In [ ]:
sp_df.head()

In [ ]:
sp_df['mean_popularity'] = [np.array(x).mean() for x in sp_df.popularity.values]

In [ ]:
float_cols = sp_df.dtypes[sp_df.dtypes == 'float64'].index.values

In [ ]:
ohecol = 'mean_popularity'

In [ ]:
sp_df['mean_popularity'].describe()

In [ ]:
sp_df['tempo'].describe()

In [ ]:
sp_df['popularitybucket'] = sp_df['mean_popularity'].apply(lambda x: int(x/5))

In [ ]:
sp_df.loc[ sp_df['tempo'] <= 30.00, 'cat_tempo'] = 0
sp_df.loc[(sp_df['tempo'] > 30.00) & (sp_df['tempo'] <= 40.00), 'cat_tempo'] = 1
sp_df.loc[(sp_df['tempo'] > 40.00) & (sp_df['tempo'] <= 60.0), 'cat_tempo']   = 2
sp_df.loc[(sp_df['tempo'] > 60.00) & (sp_df['tempo'] <= 66.00), 'cat_tempo']   = 3
sp_df.loc[(sp_df['tempo'] > 66.00) & (sp_df['tempo'] <= 80.00), 'cat_tempo']   = 4
sp_df.loc[(sp_df['tempo'] > 80.00) & (sp_df['tempo'] <= 108.00), 'cat_tempo']   = 5
sp_df.loc[(sp_df['tempo'] > 108.00) & (sp_df['tempo'] <= 124.00), 'cat_tempo']   = 6
sp_df.loc[(sp_df['tempo'] > 124.00) & (sp_df['tempo'] <= 168.00), 'cat_tempo']   = 7
sp_df.loc[(sp_df['tempo'] > 168.00) & (sp_df['tempo'] <= 176.00), 'cat_tempo']   = 8
sp_df.loc[(sp_df['tempo'] > 176.00) & (sp_df['tempo'] <= 200.00), 'cat_tempo']   = 9
sp_df.loc[ sp_df['tempo'] > 200, 'cat_tempo'] = 10
sp_df['cat_tempo'] = sp_df['cat_tempo'].astype(int)

In [ ]:
sp_df

In [ ]:
sp_df.info()

In [ ]:
def ohe_prep(df, column, new_name): 
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [ ]:
def create_feature_set(df, float_cols):
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(sp_df['genre'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names()]
    genre_df.reset_index(drop = True, inplace=True)

    tempo_ohe = ohe_prep(df, 'cat_tempo','ctempo') * 0.5
    popularity_ohe = ohe_prep(sp_df, 'popularitybucket','pbucket') * 0.15

    #scale float columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    #concanenate all features
    final = pd.concat([genre_df, floats_scaled, popularity_ohe,tempo_ohe], axis = 1)
     
    #add song id
    final['id']=sp_df['id'].values
    
    return final

In [ ]:
complete_feature_set = create_feature_set(sp_df, float_cols=float_cols)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(complete_feature_set.head(5))


In [ ]:
clientid='8bcfe563f2b4441290539a7e4a4f0d9c'
clientsecret='b01eb6f1b11840b8a1d310f2f402c4fe'

In [ ]:
scope = 'user-library-read'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

In [ ]:
auth_manager = SpotifyClientCredentials(client_id=clientid, client_secret=clientsecret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
token = util.prompt_for_user_token(scope, client_id= clientid, client_secret=clientsecret, redirect_uri='http://localhost:8881/')

In [ ]:
sp = spotipy.Spotify(auth=token)

In [ ]:
id_name = {}
list_photo = {}
for i in sp.current_user_playlists()['items']:

    id_name[i['name']] = i['uri'].split(':')[2]
    list_photo[i['uri'].split(':')[2]] = i['images'][0]['url']

In [ ]:
id_name

In [ ]:
def create_necessary_outputs(playlist_name,id_dic, df):    

    playlist = pd.DataFrame()
    playlist_name = playlist_name
    for ix, i in enumerate(sp.playlist(id_dic[playlist_name])['tracks']['items']):
       
        playlist.loc[ix, 'artist'] = i['track']['artists'][0]['name']
        playlist.loc[ix, 'name'] = i['track']['name']
        playlist.loc[ix, 'id'] = i['track']['id'] # ['uri'].split(':')[2]
        playlist.loc[ix, 'url'] = i['track']['album']['images'][1]['url']
        playlist.loc[ix, 'date_added'] = i['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
    
    playlist = playlist[playlist['id'].isin(df['id'].values)].sort_values('date_added',ascending = False)
    
    return playlist

In [ ]:
playlist_mine = create_necessary_outputs('Mixed Feelings',id_name,sp_df)

In [ ]:
from skimage import io
import matplotlib.pyplot as plt

def visualize_songs(df):

    temp = df['url'].values
    plt.figure(figsize=(15,int(0.625 * len(temp))))
    columns = 5
    
    for i, url in enumerate(temp):
        plt.subplot(len(temp) / columns + 1, columns, i + 1)

        image = io.imread(url)
        plt.imshow(image)
        plt.xticks(color = 'w', fontsize = 0.1)
        plt.yticks(color = 'w', fontsize = 0.1)
        plt.xlabel(df['name'].values[i], fontsize = 12)
        plt.tight_layout(h_pad=0.4, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    plt.show()

In [ ]:
playlist_mine

In [ ]:
visualize_songs(playlist_mine)

In [ ]:
def generate_playlist_feature(complete_feature_set, playlist_df, weight_factor):
    """ 
    Summarize a user's playlist into a single vector

    Parameters: 
        complete_feature_set (pandas dataframe): Dataframe which includes all of the features for the spotify songs
        playlist_df (pandas dataframe): playlist dataframe
        weight_factor (float): float value that represents the recency bias. The larger the recency bias, the most priority recent songs get. Value should be close to 1. 
        
    Returns: 
        playlist_feature_set_weighted_final (pandas series): single feature that summarizes the playlist
        complete_feature_set_nonplaylist (pandas dataframe): 
    """
    
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1).mean(axis =0)
    complete_feature_set_playlist = complete_feature_set_playlist.merge(playlist_df[['id','date_added']], on = 'id', how = 'inner')
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(playlist_df['id'].values)]#.drop('id', axis = 1)
    
    playlist_feature_set = complete_feature_set_playlist.sort_values('date_added',ascending=False)

    most_recent_date = playlist_feature_set.iloc[0,-1]
    
    for ix, row in playlist_feature_set.iterrows():
        playlist_feature_set.loc[ix,'months_from_recent'] = int((most_recent_date.to_pydatetime() - row.iloc[-1].to_pydatetime()).days / 30)
        
    playlist_feature_set['weight'] = playlist_feature_set['months_from_recent'].apply(lambda x: weight_factor ** (-x))
    
    playlist_feature_set_weighted = playlist_feature_set.copy()
    #print(playlist_feature_set_weighted.iloc[:,:-4].columns)
    playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-4].mul(playlist_feature_set_weighted.weight,0))
    playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-4]
    #playlist_feature_set_weighted_final['id'] = playlist_feature_set['id']
    
    return playlist_feature_set_weighted_final.sum(axis = 0), complete_feature_set_nonplaylist

In [ ]:
complete_feature_set_playlist_vector_mine, complete_feature_set_nonplaylist_mine = generate_playlist_feature(complete_feature_set, playlist_mine, 1.09)

In [ ]:
complete_feature_set_playlist_vector_mine.shape

In [ ]:
def generate_playlist_recos(df, features, nonplaylist_features):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        df (pandas dataframe): spotify dataframe
        features (pandas series): summarized playlist feature
        nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Returns: 
        non_playlist_df_top_40: Top 40 recommendations for that playlist
    """
    
    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    non_playlist_df_top_40['url'] = non_playlist_df_top_40['id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return non_playlist_df_top_40

In [ ]:
mine_top40 = generate_playlist_recos(sp_df, complete_feature_set_playlist_vector_mine, complete_feature_set_nonplaylist_mine)